In [ ]:
from pasteur.kedro.ipython import * # type: ignore
register_kedro() # type: ignore

INFO     Kedro project Pasteur                                                                                           __init__.py:77
INFO     Defined global variable 'context', 'session', 'catalog' and 'pipelines'                                         __init__.py:78
WARNING  /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/plotly/graph_objects/__init__.py:288:              warnings.py:109
           if LooseVersion(ipywidgets.__version__) >= LooseVersion("7.0.0"):                                                           
                                                                                                                                       
INFO     Registered line magic 'run_viz'                                                                                 __init__.py:84


In [ ]:
%load_ext autoreload
%autoreload 2

WARNING  /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/extensions/autoreload.py:121:              warnings.py:109
         see the module's documentation for alternative uses                                                                           
           from imp import reload                                                                                                      
                                                                                                                                       


In [ ]:
view = "texas_charges"

# LIM = 100000
enc = catalog.load(f'{view}.wrk.idx_table').sample()#[:LIM]
trn = catalog.load(f'{view}.trn.table')
attrs = trn['idx'].get_attributes()
len(enc)

9251363


In [ ]:
from pasteur.extras.synth.math import expand_table, calc_marginal, AttrSelector, AttrSelectors, get_dtype

cols, cols_noncommon, domains = expand_table(attrs, enc)
domains.keys()

dict_keys(['revenue_code', 'hcpcs_qualifier', 'hcpcs_procedure_code', 'modifier_1', 'modifier_2', 'modifier_3', 'modifier_4', 'unit_measurement_code', 'units_of_service', 'unit_rate', 'chrgs_line_item', 'chrgs_non_cov'])


In [ ]:
tst = ['unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code','unit_measurement_code','unit_measurement_code']
# tst = ['unit_measurement_code', 'unit_measurement_code', 'unit_measurement_code']

In [ ]:
x = AttrSelector(tst[0], 0, {tst[0]: 0})
p = {str(i): AttrSelector(col, 0, {col: 0}) for i, col in enumerate(tst[1:])}

%timeit calc_marginal(cols, cols_noncommon, domains, x, p)

78.7 ms ± 6.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
import numpy as np

%timeit np.histogramdd(enc[tst].to_numpy(), [domains[col][0] for col in tst])

783 ms ± 2.34 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%load_ext line_profiler

In [ ]:

def calc_marginal2(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelector,
    p: AttrSelectors,
    partial: bool = False,
    zero_fill: float | None = None,
):
    """Calculates the 1 way and 2 way marginals between the subsection of the
    hierarchical attribute x and the attributes p(arents)."""

    # Find integer dtype based on domain
    p_dom = 1
    for attr in p.values():
        common = attr.common
        l_dom = 1
        for i, (n, h) in enumerate(attr.cols.items()):
            l_dom *= domains[n][h] - common
        p_dom *= l_dom + common
    x_dom = 1
    for i, (n, h) in enumerate(x.cols.items()):
        x_dom *= domains[n][h] - x.common
    x_dom += x.common

    dtype = get_dtype(p_dom * x_dom)

    n = len(next(iter(cols.values()))[0])
    _sum_nd = np.zeros((n,), dtype=dtype)
    _tmp_nd = np.zeros((n,), dtype=dtype)

    # Handle parents
    mul = 1
    for attr_name, attr in p.items():
        common = attr.common
        l_mul = 1
        p_partial = partial and attr_name == x.name
        for i, (n, h) in enumerate(attr.cols.items()):
            if common == 0 or i == 0:
                np.multiply(cols[n][h], mul * l_mul, out=_tmp_nd, dtype=dtype)
            else:
                np.multiply(cols_noncommon[n][h], mul * l_mul, out=_tmp_nd, dtype=dtype)

            np.add(_sum_nd, _tmp_nd, out=_sum_nd, dtype=dtype)
            l_mul *= domains[n][h] - common

        if p_partial:
            mul *= l_mul
        else:
            mul *= l_mul + common

    # Handle x
    common = x.common
    for i, (n, h) in enumerate(x.cols.items()):
        if common == 0 or (i == 0 and not partial):
            np.multiply(cols[n][h], mul, out=_tmp_nd, dtype=dtype)
        else:
            np.multiply(cols_noncommon[n][h], mul, out=_tmp_nd, dtype=dtype)

        np.add(_sum_nd, _tmp_nd, out=_sum_nd, dtype=dtype)
        mul *= domains[n][h] - common

    # Keep only non-common items if there is a parent to source the others
    if partial:
        n = next(iter(x.cols))
        _sum_nd = _sum_nd[cols[n][0] >= common]
        x_dom = x_dom - x.common

    return np.bincount(_sum_nd, minlength=p_dom * x_dom)#.reshape(x_dom, p_dom)
    # return np.zeros()

    # j_mar = margin
    # x_mar = np.sum(margin, axis=1)
    # p_mar = np.sum(margin, axis=0)

    # return j_mar, x_mar, p_mar

%timeit calc_marginal2(cols, cols_noncommon, domains, x, p)
# %lprun -f calc_marginal2 calc_marginal2(cols, cols_noncommon, domains, x, p)
print(calc_marginal2(cols, cols_noncommon, domains, x, p)[:10])

95.7 ms ± 92.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
[210   0   0   0   0   0   0   0   0   0]


In [ ]:
import jax.numpy as jnp
from jax import grad, jit, vmap, default_device, devices
from jax import random, ops
from typing import NamedTuple
import jax.lax as lax


class JaxSelector(NamedTuple):
    common: int
    cols: list[tuple[int, jnp.ndarray]]


JaxSelectors = list[JaxSelector]


def collapse_marginal(x: JaxSelector, P: JaxSelectors):
    # Handle parents
    sum_arr = None
    mul = 1
    for p in P:
        common = p.common
        l_mul = 1
        for dom, col in p.cols:
            tmp_arr = col * (mul * l_mul)
            if sum_arr is None:
                sum_arr = tmp_arr
            else:
                sum_arr += tmp_arr
            l_mul *= dom - common

    # Handle x
    assert sum_arr is not None
    common = x.common
    for dom, col in x.cols:
        sum_arr += dom * mul
        mul *= dom - common

    return sum_arr

2000-01-01 00:00:00,000752: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2000-01-01 00:00:00,000143: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2000-01-01 00:00:00,000532: W external/org_tensorflow/tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [ ]:
def count(sum_arr, length: int):
    # return jnp.bincount(sum_arr, length=length)
    # return lax.broadcast(jnp.uint32(0), [length]).at[sum_arr].add(jnp.uint8(1))
    def f(carry, x):
        return carry, x
    
    return lax.scan(f, lax.broadcast(jnp.uint16(0), [length]), sum_arr)
    # op = lax.broadcast(jnp.uint16(0), [length])
    # sc = lax.expand_dims(sum_arr, [1])
    # up = lax.broadcast(jnp.uint16(1), [sc.shape[0]])
    # return lax.scatter_add(op, sc, up, lax.ScatterDimensionNumbers(tuple(), (0,), (0,)), mode="promise_in_bounds")

def fun():
    x = JaxSelector(0, [(domains[tst[0]][0], jnp.array(cols[tst[0]][0]))])
    P = []
    for col in tst[1:]:
        P.append(JaxSelector(0, [(domains[col][0], jnp.array(cols[col][0]))]))
    
    p_dom = 1
    for p in P:
        common = p.common
        l_dom = 1
        for dom, _ in p.cols:
            l_dom *= dom - common
        p_dom *= l_dom + common
    x_dom = 1
    for dom, _ in x.cols:
        x_dom *= dom - x.common
    x_dom += x.common
    length = p_dom * x_dom
    print(length)

    try:
        cmp = jit(collapse_marginal).lower(x, P)
        # print(cmp.as_text())
        cmp = cmp.compile()
        # print(cmp.as_text())

        print(jit(count, static_argnums=1).lower(cmp(x, P), length).as_text())
        print(count(cmp(x, P), length)[:10])
    except Exception as e:
        print(e)
        return

    TIMES = 10
    def xn():
        out = count(cmp(x, P), length)

        for i in range(TIMES-1):
            out += count(cmp(x, P), length)
        
        out.block_until_ready()

    # %timeit xn()
    %timeit count(cmp(x, P), length).block_until_ready()
    %timeit xn()


for cuda in (False,): #True):
    d = devices('cuda' if cuda else 'cpu')[0]
    with default_device(d):
        fun()


INFO     Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker:        xla_bridge.py:353
INFO     Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm".        xla_bridge.py:353
         Available platform names are: Host Interpreter CUDA                                                                           
INFO     Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'          xla_bridge.py:353
INFO     Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client.       xla_bridge.py:353
         Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to                        
         false) to enable this.                                                                                                        
46656
module @jit_count {
  func.func public @ma

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_77997/3874527509.py:62 in <cell line: 59>                                         │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_77997/3874527509.py'                        │
│                                                                                                  │
│ /tmp/ipykernel_77997/3874527509.py:55 in fun                                                     │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_77997/3874527509.py'                        │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:241 │
│ 8 in run_line_magic                                                                              │
│                                                                                                  │
│   2415 │   │   │   if getattr(fn, "needs_local_scope", False):                                   │
│   2416 │   │   │   │   kwargs['local_ns'] = self.get_local_scope(stack_depth)                    │
│   2417 │   │   │   with self.builtin_trap:                                                       │
│ ❱ 2418 │   │   │   │   result = fn(*args, **kwargs)                                              │
│   2419 │   │   │   return result                                                                 │
│   2420 │                                                                                         │
│   2421 │   def get_local_scope(self, stack_depth):                                               │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/decorator.py:232 in fun              │
│                                                                                                  │
│   229 │   │   def fun(*args, **kw):                                                              │
│   230 │   │   │   if not kwsyntax:                                                               │
│   231 │   │   │   │   args, kw = fix(args, kw, sig)                                              │
│ ❱ 232 │   │   │   return caller(func, *(extras + args), **kw)                                    │
│   233 │   fun.__name__ = func.__name__                                                           │
│   234 │   fun.__doc__ = func.__doc__                                                             │
│   235 │   fun.__wrapped__ = func                                                                 │
│                                                                                                  │
│ /mnt/ext/projects/pasteur/venv/lib/python3.10/site-packages/IPython/core/magic.py:187 in         │
│ <lambda>                                                                                         │
│                                                                                                  │
│   184 │   # This is a closure to capture the magic_kind.  We could also use a class,             │
│   185 │   # but it's overkill for just that one bit of state.                                    │
│   186 │   def magic_deco(arg):                                                                   │
│ ❱ 187 │   │   call = lambda f, *a, **k: f(*a, **k)                                               │
│   188 │   │                                                                                      │
│   189 │   │   if callable(arg):                                                                  │
│   190 │   │   │   # "Naked" decorator call (just @foo, no a

In [ ]:
%load_ext cython

In [ ]:
%%cython -a
import cython as cy

import numpy as np
cimport numpy as np

np.import_array()

@cy.inline
@cy.boundscheck(False)
@cy.wraparound(False)
cdef cython_inner_inline(
    int l, np.ndarray[np.uint32_t, ndim=1] out, 
    int n_u8,  int* m_u8,  np.ndarray[np.uint8_t,  ndim=1]*  p_u8, 
    int n_u16, int* m_u16, np.ndarray[np.uint16_t, ndim=1]* p_u16,
    int n_u32, int* m_u32, np.ndarray[np.uint32_t, ndim=1]* p_u32, 
):
    cdef int idx

    for i in range(l):
        idx = 0
        for j in range(n_u8):
            idx += m_u8[j] * p_u8[j][i]
        for j in range(n_u16):
            idx += m_u16[j] * p_u16[j][i]
        for j in range(n_u32):
            idx += m_u32[j] * p_u32[j][i]
        
        out[idx] += 1
    return out

def cython_inner_wrapper(
    dom: int, l: int,
    out: np.ndarray | None,
    ops: tuple[int, np.ndarray]
):
    if not out:
        out = np.zeros((dom,), dtype=np.uint32)
    
    # l = len(ops)
    # cdef int[l] m_


Error compiling Cython file:
------------------------------------------------------------
...
@cy.inline
@cy.boundscheck(False)
@cy.wraparound(False)
cdef cython_inner_inline(
    int l, np.ndarray[np.uint32_t, ndim=1] out, 
    int n_u8,  int* m_u8,  np.ndarray[np.uint8_t,  ndim=1]*  p_u8, 
                                                         ^
------------------------------------------------------------

/home/dev/.cache/ipython/cython/_cython_magic_82a1b7da1352fe47ca8eebd94dea137b.pyx:13:58: Pointer base type cannot be a Python object

Error compiling Cython file:
------------------------------------------------------------
...
@cy.boundscheck(False)
@cy.wraparound(False)
cdef cython_inner_inline(
    int l, np.ndarray[np.uint32_t, ndim=1] out, 
    int n_u8,  int* m_u8,  np.ndarray[np.uint8_t,  ndim=1]*  p_u8, 
    int n_u16, int* m_u16, np.ndarray[np.uint16_t, ndim=1]* p_u16,
                                                         ^
------------------------------------------

In [ ]:
def numba_inner(dom: int, l: int, out: np.ndarray, *o: Op):
    assert len(o) == 6, f'Length is {len(o)}'
    return numba_inner_6(
        dom,
        l,
        out,
        o[0][0],
        o[0][1],
        o[1][0],
        o[1][1],
        o[2][0],
        o[2][1],
        o[3][0],
        o[3][1],
        o[4][0],
        o[4][1],
        o[5][0],
        o[5][1],
    )

def calc_marginal_numba(
    cols: dict[str, list[np.ndarray]],
    cols_noncommon: dict[str, list[np.ndarray]],
    domains: dict[str, list[int]],
    x: AttrSelector,
    p: AttrSelectors,
    partial: bool = False,
    zero_fill: float | None = None,
):
    """Calculates the 1 way and 2 way marginals between the subsection of the
    hierarchical attribute x and the attributes p(arents)."""

    l = len(next(iter(cols.values()))[0])
    # Handle parents
    ops = []
    mul = 1
    for attr_name, attr in p.items():
        common = attr.common
        l_mul = 1
        p_partial = partial and attr_name == x.name
        for i, (n, h) in enumerate(attr.cols.items()):
            if common == 0 or i == 0:
                ops.append((mul*l_mul, cols[n][h]))
            else:
                ops.append((mul*l_mul, cols_noncommon[n][h]))

            l_mul *= domains[n][h] - common

        if p_partial:
            mul *= l_mul
        else:
            mul *= l_mul + common
    p_dom = mul

    # Handle x
    common = x.common
    for i, (n, h) in enumerate(x.cols.items()):
        if common == 0 or (i == 0 and not partial):
            ops.append((mul*l_mul, cols[n][h]))
        else:
            ops.append((mul*l_mul, cols_noncommon[n][h]))

        mul *= domains[n][h] - common
    x_dom = mul
    dom = x_dom * p_dom

    # FIXME: partial support
    # Keep only non-common items if there is a parent to source the others
    # if partial:
    #     n = next(iter(x.cols))
    #     _sum_nd = _sum_nd[cols[n][0] >= common]
    #     x_dom = x_dom - x.common

    return numba_inner(dom, l, None, *ops)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_301404/3250864887.py:1 in <cell line: 1>                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_301404/3250864887.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'Op' is not defined

In [ ]:
calc_marginal_numba(cols, cols_noncommon, domains, x, p)
%timeit calc_marginal_numba(cols, cols_noncommon, domains, x, p)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_301404/1765432453.py:1 in <cell line: 1>                                          │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_301404/1765432453.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'calc_marginal_numba' is not defined